In [1]:
from ase.io import read, write
import ase
from phonopy import Phonopy
from phonopy.structure.atoms import PhonopyAtoms
import numpy as np
import phonopy

In [95]:
def get_phonon_configs(calc, at, disps, scell, config_type):

    cell = PhonopyAtoms(symbols=at.get_chemical_symbols(),
                    cell=at.get_cell(),
                    positions=at.get_positions())

    phonon = Phonopy(cell, np.eye(3)*scell)

    al = []

    for disp in disps:
        phonon.generate_displacements(distance=disp)
        supercells = phonon.get_supercells_with_displacements()

        for (i,scell) in enumerate(supercells):
            at = ase.Atoms(symbols=scell.get_chemical_symbols(),
                      scaled_positions=scell.get_scaled_positions(),
                      cell=scell.get_cell(),
                      pbc=True)

            at.set_calculator(calc)

            energy = at.get_potential_energy()#force_consistent=True)
            forces = at.get_forces()
            #stress = at.get_stress(voigt=False)

            at.arrays["force"] = forces
            at.info["energy"] = energy
            at.info["config_type"] = "PH_" + config_type
            #write("PH_{}_{}_scell_{}.xyz".format(config_type, i, disp), at)
            al.append(at)

    return al

In [88]:
def get_FLD_configs(calc, in_at, config_type, Vs=np.linspace(0.94, 1.06, 10), Ds=np.linspace(-0.24, 0.24, 10)):
    import copy

    al = []
    for i in Vs:
        for j in Ds:
            at = copy.deepcopy(in_at)
            k, l = np.random.randint(0,3), np.random.randint(0,3)
            at.set_cell(at.cell * i, scale_atoms=True)
            at.positions = at.positions + np.random.normal(0, 0.2, (len(at),3))
            at.cell[k][l] += (j + np.random.normal(0, 0.2))
            at.set_calculator(calc)
            at.info["energy"] = at.get_potential_energy()
            at.arrays["force"] = at.get_forces()
            #at.info["virial"] = -1.0 * at.get_volume() * at.get_stress(voigt=False)
            at.info["config_type"] = config_type
            #write("hcp_FLD_{}_{}_{}_{}.xyz".format(i,j,k,l), at)
            al.append(at)
    
    return al

In [89]:
from ase.calculators.eam import EAM

calc = EAM(potential='Ti1.eam.fs')

In [2]:
from ase.lattice.cubic import BodyCenteredCubic
from ase.lattice.hexagonal import HexagonalClosedPacked

at = BodyCenteredCubic("Ti", latticeconstant=3.32)

al_bcc = get_FLD_configs(calc, at, "bcc")

NameError: name 'get_FLD_configs' is not defined

In [83]:
E = [at.get_potential_energy()/len(at) for at in al_bcc]
V = [at.get_volume()/len(at) for at in al_bcc]

In [84]:
import matplotlib.pyplot as plt

plt.scatter(V,E)

In [97]:
PH_config = get_phonon_configs(calc, at, [0.02], 3, "bcc")

In [105]:
PH_config[0].arrays

{'numbers': array([22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
        22, 22, 22]),
 'positions': array([[0.02, 0.  , 0.  ],
        [3.32, 0.  , 0.  ],
        [6.64, 0.  , 0.  ],
        [0.  , 3.32, 0.  ],
        [3.32, 3.32, 0.  ],
        [6.64, 3.32, 0.  ],
        [0.  , 6.64, 0.  ],
        [3.32, 6.64, 0.  ],
        [6.64, 6.64, 0.  ],
        [0.  , 0.  , 3.32],
        [3.32, 0.  , 3.32],
        [6.64, 0.  , 3.32],
        [0.  , 3.32, 3.32],
        [3.32, 3.32, 3.32],
        [6.64, 3.32, 3.32],
        [0.  , 6.64, 3.32],
        [3.32, 6.64, 3.32],
        [6.64, 6.64, 3.32],
        [0.  , 0.  , 6.64],
        [3.32, 0.  , 6.64],
        [6.64, 0.  , 6.64],
        [0.  , 3.32, 6.64],
        [3.32, 3.32, 6.64],
        [6.64, 3.32, 6.64],
        [0.  , 6.64, 6.64],
        [3.32, 6.64, 6.64]

In [14]:
at = BodyCenteredCubic("Ti", latticeconstant=3.32) * (3,3,3)
at.info["config_type"] = "bcc"
write("./bcc.xyz", at)

In [5]:
big_at.

Lattice(symbols='Ti54', pbc=True, cell=[9.959999999999999, 9.959999999999999, 9.959999999999999])